In [1]:
library(knitr)
library(rmdformats)
library(data.table)
library(magrittr)
library(lubridate)
library(ggplot2)
library(cowplot)
library(repr)
library(kableExtra)
library(IRdisplay)
print("All packages loaded successfully")


Attaching package: 'lubridate'


The following objects are masked from 'package:data.table':

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from 'package:base':

    date, intersect, setdiff, union



Attaching package: 'cowplot'


The following object is masked from 'package:lubridate':

    stamp




[1] "All packages loaded successfully"


In [2]:
simulate_central_scenario <- function(seed = 1234){
  
  #seed = 1234  
  set.seed(seed)
  
  # Policy data
  #~~~~~~~~~~~~~~~~~
  
  # polices sold between start 2016 to end 2017
  dt_policydates <- data.table(date_UW = seq(as.Date("2016/1/1"), as.Date("2017/12/31"), "day"))
  
  # number of policies per day follows Poisson process with mean 700 (approx 255,500 pols per annum)
  dt_policydates[, ':='(policycount = rpois(.N,700),
                        date_lapse = date_UW %m+% years(1),
                        expodays = as.integer(date_UW %m+% years(1) - date_UW),
                        pol_prefix = year(date_UW)*10000 + month(date_UW)*100 + mday(date_UW))]
  
  
  # Add columns defining Policy Covers   
  dt_policydates[, Cover_B := round(policycount * 0.25)]
  dt_policydates[, Cover_BO := round(policycount * 0.45)]
  dt_policydates[, Cover_BOT := policycount - Cover_B - Cover_BO]
  
  
  # repeat rows for each policy by UW-Date
  dt_policy <- dt_policydates[rep(1:.N, policycount),c("date_UW", "pol_prefix"), with = FALSE][,pol_seq:=1:.N, by=pol_prefix]
  
  # Create a unique policy number 
  dt_policy[, pol_number := as.character(pol_prefix * 10000 + pol_seq)]
  
  # set join keys
  setkey(dt_policy,'date_UW')
  setkey(dt_policydates,'date_UW')  
  
  # remove pol_prefix before join
  dt_policydates[, pol_prefix := NULL]  
  
  # join cover from summary file (dt_policydates)
  dt_policy <- dt_policy[dt_policydates]  
  
  # now create Cover field for each policy row
  dt_policy[,Cover := 'BO']
  dt_policy[pol_seq <= policycount- Cover_BO,Cover := 'BOT']
  dt_policy[pol_seq <= Cover_B,Cover := 'B']  
  
  # remove interim calculation fields
  dt_policy[, ':='(pol_prefix = NULL,
                   policycount = NULL,
                   pol_seq = NULL,
                   Cover_B = NULL,
                   Cover_BOT = NULL,
                   Cover_BO = NULL)]
  
  # Add remaining policy details
  dt_policy[, Brand := rep(rep(c(1,2,3,4), c(9,6,3,2)), length.out = .N)]
  dt_policy[, Base_Price := rep(rep(c(600,550,300,150), c(9,6,3,2)), length.out = .N)]
  
  # models types and model cost multipliers
  for (eachBrand in unique(dt_policy$Brand)) {
    dt_policy[Brand == eachBrand, Model := rep(rep(c(3,2,1,0), c(10, 7, 2, 1)), length.out = .N)]
    dt_policy[Brand == eachBrand, Model_mult := rep(rep(c(1.15^3, 1.15^2, 1.15^1, 1.15^0), c(10, 7, 2, 1)), length.out = .N)]
  }
  
  dt_policy[, Price := ceiling (Base_Price * Model_mult)]
  
  
  # colums to keep
  cols_policy <- c("pol_number",
                   "date_UW",
                   "date_lapse",
                   "Cover",
                   "Brand",
                   "Model",
                   "Price")
  
  dt_policy <- dt_policy[, cols_policy, with = FALSE]
  
  # check output
  head(dt_policy)
  
  #save(dt_policy, file = "./dt_policy.rda")
  
  
  # Claims data
  #~~~~~~~~~~~~~~~~~
  
 # All policies have breakage cover
  # claims uniformly sampled from policies
  claim <- sample(nrow(dt_policy), size = floor(nrow(dt_policy) * 0.15))
  
  # Claim serverity multiplier sampled from beta distn
  dt_claim <- data.table(pol_number = dt_policy[claim, pol_number],
                         claim_type = 'B',
                         claim_count = 1,
                         claim_sev = rbeta(length(claim), 2,5))
  
  # identify all policies with Oxidation cover
  cov <- which(dt_policy$Cover != 'B')
  
  # sample claims from policies with cover
  claim <- sample(cov, size = floor(length(cov) * 0.05))
  
  # add claims to table 
  dt_claim <- rbind(dt_claim,
                    data.table(pol_number = dt_policy[claim, pol_number],
                               claim_type = 'O',
                               claim_count = 1,
                               claim_sev = rbeta(length(claim), 5,3)))
  
  
  # identify all policies with Theft cover
  # for Theft claim frequency varies by Brand
  # So need to consider each in turn...
  
  for(myModel in 0:3) {
    
    cov <- which(dt_policy$Cover == 'BOT' & dt_policy$Model == myModel)
    claim <- sample(cov, size = floor(length(cov) * 0.05*(1 + myModel)))
    
    dt_claim <- rbind(dt_claim,
                      data.table(pol_number = dt_policy[claim, pol_number],
                                 claim_type = 'T',
                                 claim_count = 1,
                                 claim_sev = rbeta(length(claim), 5,.5)))
  }
  
  # set join keys
  setkey(dt_policy, pol_number)
  setkey(dt_claim, pol_number)
  
  #join Brand and Price from policy to claim
  dt_claim[dt_policy,
           on = 'pol_number',
           ':='(date_UW = i.date_UW,
                Price = i.Price,
                Brand = i.Brand)]
  
  # use lubridate %m+% date addition operator 
  dt_claim[, date_lapse := date_UW %m+% years(1)]
  dt_claim[, expodays := as.integer(date_lapse - date_UW)]
  dt_claim[, occ_delay_days := floor(expodays * runif(.N, 0,1))]
  
  dt_claim[ ,delay_report := floor(365 * rbeta(.N, .4, 10))]  
  dt_claim[ ,delay_pay := floor(10 + 40* rbeta(.N, 7,7))]  
  
  dt_claim[, date_occur := date_UW %m+% days(occ_delay_days)]
  dt_claim[, date_report := date_occur %m+% days(delay_report)]
  dt_claim[, date_pay := date_report %m+% days(delay_pay)]
  
  dt_claim[, claim_cost := round(Price * claim_sev)]
  
  dt_claim[, clm_prefix := year(date_report)*10000 + month(date_report)*100 + mday(date_report)]
  
  dt_claim[, clm_seq := seq_len(.N), by = clm_prefix]
  dt_claim[, clm_number := as.character(clm_prefix * 10000 + clm_seq)]
  
  # colums to keep
  cols_claim <- c("clm_number",
                  "pol_number",
                  "claim_type",
                  "claim_count",
                  "claim_sev",
                  "date_occur",
                  "date_report",
                  "date_pay",
                  "claim_cost")
  
  dt_claim <- dt_claim[, cols_claim, with = FALSE]
  
 output <- list()
 output$dt_policy <- dt_policy
 output$dt_claim <- dt_claim

 return(output)

  
  
}

In [3]:

dt_PhoneData <- simulate_central_scenario(1234)

In [4]:

dt_policy <- dt_PhoneData$dt_policy

kable(head(dt_policy), "html") %>% 
  kable_styling("striped") %>% 
  scroll_box(width = "100%") %>%
  as.character() %>%
  display_html()
     

pol_number,date_UW,date_lapse,Cover,Brand,Model,Price
201601010001,2016-01-01,2017-01-01,B,1,3,913
201601010002,2016-01-01,2017-01-01,B,1,3,913
201601010003,2016-01-01,2017-01-01,B,1,3,913
201601010004,2016-01-01,2017-01-01,B,1,3,913
201601010005,2016-01-01,2017-01-01,B,1,3,913
201601010006,2016-01-01,2017-01-01,B,1,3,913


In [5]:
dt_claim <- dt_PhoneData$dt_claim

kable(head(dt_claim), "html") %>% 
  kable_styling(c("striped", "hover", "condensed")) %>% 
  scroll_box(width = "100%") %>%
  as.character() %>%
  display_html()
     

clm_number,pol_number,claim_type,claim_count,claim_sev,date_occur,date_report,date_pay,claim_cost
201606080001,201601010001,B,1,0.3337923,2016-06-08,2016-06-08,2016-07-21,305
201609150001,201601010014,B,1,0.3692034,2016-09-15,2016-09-15,2016-10-17,309
201609090001,201601010025,B,1,0.4496012,2016-09-09,2016-09-09,2016-10-07,357
201602190001,201601010027,B,1,0.4019731,2016-01-25,2016-02-19,2016-03-21,319
201605140001,201601010043,B,1,0.2146653,2016-05-14,2016-05-14,2016-06-15,196
201612110001,201601010045,B,1,0.2783313,2016-12-11,2016-12-11,2017-01-06,254


In [6]:
setnames(dt_policy, c('date_UW', 'date_lapse'), c('date_pol_start', 'date_pol_end'))
  
# set policy start and end dates in foverlap friendly format
dt_policy[, date_pol_start:= floor_date(date_pol_start, unit= "second")]
dt_policy[, date_pol_end:= floor_date(date_pol_end, unit= "second") - 1]
  
# create a dummy end claim occurrence date for foverlap
dt_claim[, date_occur:= floor_date(date_occur, unit= "second")]
dt_claim[, date_occur_end:= date_occur]
dt_claim[, date_report:= floor_date(date_report, unit= "second")]
dt_claim[, date_pay:= floor_date(date_pay, unit= "second")]
  
# set keys for claim join (by policy and dates)
setkey(dt_claim, pol_number, date_occur, date_occur_end)
setkey(dt_policy, pol_number, date_pol_start, date_pol_end)
  
# use foverlaps to attach claim to right occurrence period and policy
dt_polclaim <- foverlaps(dt_policy, dt_claim, type="any") ## return overlap indices
dt_polclaim[, date_occur_end := NULL]

In [7]:

head(dt_polclaim)

pol_number,clm_number,claim_type,claim_count,claim_sev,date_occur,date_report,date_pay,claim_cost,date_pol_start,date_pol_end,Cover,Brand,Model,Price
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dbl>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>
201601010001,201606080001,B,1,0.3337923,2016-06-08,2016-06-08,2016-07-21,305,2016-01-01,2016-12-31 23:59:59,B,1,3,913
201601010002,NA,NA,NA,NA,NA,NA,NA,NA,2016-01-01,2016-12-31 23:59:59,B,1,3,913
201601010003,NA,NA,NA,NA,NA,NA,NA,NA,2016-01-01,2016-12-31 23:59:59,B,1,3,913
201601010004,NA,NA,NA,NA,NA,NA,NA,NA,2016-01-01,2016-12-31 23:59:59,B,1,3,913
201601010005,NA,NA,NA,NA,NA,NA,NA,NA,2016-01-01,2016-12-31 23:59:59,B,1,3,913
201601010006,NA,NA,NA,NA,NA,NA,NA,NA,2016-01-01,2016-12-31 23:59:59,B,1,3,913


In [8]:


setkey(dt_polclaim, pol_number, date_pol_start)
  
# create 2 new cols that count how many claims against each policy
dt_polclaim[,
            ':='(pol_seq = seq_len(.N),
                 pol_seq_max = .N),
            by = c('pol_number', 'date_pol_start') ]
  
table(dt_polclaim[, pol_seq_max])


     1      2      3 
504688  14750    549 

In [9]:
#set NA dates to 31/12/2999
lst_datefields <- grep(names(dt_polclaim),pattern = "date", value = TRUE)
  
for (datefield in lst_datefields)
  set(dt_polclaim,which(is.na(dt_polclaim[[datefield]])),datefield,as_datetime("2199-12-31 23:59:59 UTC"))
 
#set other NAs to zero (claim counts and costs)
for (field in c("claim_count", "claim_sev", "claim_cost"))
  set(dt_polclaim,which(is.na(dt_polclaim[[field]])),field,0)

In [10]:
#Clean up ----
dt_polclaim[, ExpoDays:= ceiling((as.numeric(date_pol_end) - as.numeric(date_pol_start))/(24*60*60*365))]
dt_polclaim <- dt_polclaim[ExpoDays > 0]
  
rm(dt_claim)
rm(dt_policy)
  
gc()

     

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,1555055,83.1,3424430,182.9,3424430,182.9
Vcells,16515594,126.1,28010710,213.8,27936672,213.2


In [11]:

head(dt_polclaim)

pol_number,clm_number,claim_type,claim_count,claim_sev,date_occur,date_report,date_pay,claim_cost,date_pol_start,date_pol_end,Cover,Brand,Model,Price,pol_seq,pol_seq_max,ExpoDays
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dbl>,<dttm>,<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
201601010001,201606080001,B,1,0.3337923,2016-06-08 00:00:00,2016-06-08 00:00:00,2016-07-21 00:00:00,305,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2
201601010002,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2
201601010003,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2
201601010004,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2
201601010005,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2
201601010006,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,2016-12-31 23:59:59,B,1,3,913,1,1,2


In [12]:

lst_Date_slice <- floor_date(seq(as.Date("2016/1/1"), as.Date("2019/06/30"), by = 30), unit= "second") 

# Time slice Policy & claims 
 
for (i in 1:length(lst_Date_slice)){
  dt_polclaim[date_pay<= lst_Date_slice[i], paste0('P_t_', format(lst_Date_slice[i], "%Y%m%d")):= claim_cost]
  set(dt_polclaim,which(is.na(dt_polclaim[[paste0('P_t_', format(lst_Date_slice[i], "%Y%m%d"))]])),paste0('P_t_', format(lst_Date_slice[i], "%Y%m%d")),0)
}
  
# sort data by policynumber
setkey(dt_polclaim, pol_number)

In [13]:

head(dt_polclaim)

pol_number,clm_number,claim_type,claim_count,claim_sev,date_occur,date_report,date_pay,claim_cost,date_pol_start,...,P_t_20180917,P_t_20181017,P_t_20181116,P_t_20181216,P_t_20190115,P_t_20190214,P_t_20190316,P_t_20190415,P_t_20190515,P_t_20190614
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dbl>,<dttm>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201601010001,201606080001,B,1,0.3337923,2016-06-08 00:00:00,2016-06-08 00:00:00,2016-07-21 00:00:00,305,2016-01-01,...,305,305,305,305,305,305,305,305,305,305
201601010002,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,...,0,0,0,0,0,0,0,0,0,0
201601010003,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,...,0,0,0,0,0,0,0,0,0,0
201601010004,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,...,0,0,0,0,0,0,0,0,0,0
201601010005,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,...,0,0,0,0,0,0,0,0,0,0
201601010006,NA,NA,0,0.0000000,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,0,2016-01-01,...,0,0,0,0,0,0,0,0,0,0


In [48]:
total_rows <- nrow(dt_polclaim)
cat("Total number of rows:", total_rows, "\n")

Total number of rows: 519987 


In [50]:
dt_polclaim_with_claim_no <- dt_polclaim[!is.na(clm_number)]
head(dt_polclaim_with_claim_no)

total_rows <- nrow(dt_polclaim_with_claim_no)
cat("Total number of rows:", total_rows, "\n")

pol_number,clm_number,claim_type,claim_count,claim_sev,date_occur,date_report,date_pay,claim_cost,date_pol_start,...,P_t_20180917,P_t_20181017,P_t_20181116,P_t_20181216,P_t_20190115,P_t_20190214,P_t_20190316,P_t_20190415,P_t_20190515,P_t_20190614
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dbl>,<dttm>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201601010001,201606080001,B,1,0.3337923,2016-06-08,2016-06-08,2016-07-21,305,2016-01-01,...,305,305,305,305,305,305,305,305,305,305
201601010014,201609150001,B,1,0.3692034,2016-09-15,2016-09-15,2016-10-17,309,2016-01-01,...,309,309,309,309,309,309,309,309,309,309
201601010025,201609090001,B,1,0.4496012,2016-09-09,2016-09-09,2016-10-07,357,2016-01-01,...,357,357,357,357,357,357,357,357,357,357
201601010027,201602190001,B,1,0.4019731,2016-01-25,2016-02-19,2016-03-21,319,2016-01-01,...,319,319,319,319,319,319,319,319,319,319
201601010043,201605140001,B,1,0.2146653,2016-05-14,2016-05-14,2016-06-15,196,2016-01-01,...,196,196,196,196,196,196,196,196,196,196
201601010045,201612110001,B,1,0.2783313,2016-12-11,2016-12-11,2017-01-06,254,2016-01-01,...,254,254,254,254,254,254,254,254,254,254


Total number of rows: 121402 


In [14]:

#_ 2.1 Set initial variables ----
#~~~~~~~~~~~~~~~~~~~~~~~
    
i <- valuation <- 10
t_i <- lst_Date_slice[i] 
delta <- min(i, length(lst_Date_slice) - i + 1)
     

In [15]:

RBNS_Train_ijk <- function(dt_policy_claim, date_i, j_dev_period, k, reserving_dates, model_vars) {
  
  # # debugging
  # #~~~~~~~~~~~~~
  # dt_policy_claim = dt_polclaim
  # date_i = t_i
  # j_dev_period = 1
  # k = 1
  # reserving_dates = lst_Date_slice
  # #~~~~~~~~~~~~~
  #   
  
  date_i <- as.Date(date_i)
  date_k <- (reserving_dates[which(reserving_dates == date_i) - k + 1])
  date_j <- (reserving_dates[which(reserving_dates == date_k) - j_dev_period])
  
  #i - j - k + 1 (predictor as at date)
  date_lookup <- (reserving_dates[which(reserving_dates == (date_i)) - j_dev_period -k + 1]) 
  
  #i - k to calculate target incremental paid
  target_lookup <- (reserving_dates[which(reserving_dates == (date_i)) - k]) 
  
  #i -k + 1 to calculate target incremental paid
  target_lookup_next <- (reserving_dates[which(reserving_dates == (date_i)) - k + 1]) 
  
  #definition of reported but not settled
  dt_policy_claim <- dt_policy_claim[(date_report <= date_lookup) & (date_pay > date_lookup)] 
  
  #simulated data assumes one payment so just need to check date paid in target calc
  dt_policy_claim[, ':='(date_lookup = date_lookup,
                         delay_train = as.numeric(date_lookup - date_pol_start), #extra feature
                         j = j_dev_period,
                         k = k,
                         target = ifelse(date_pay<=target_lookup,0,ifelse(date_pay<=target_lookup_next,claim_cost,0)))]
  
  return(dt_policy_claim[, model_vars, with = FALSE])
}


In [16]:
RBNS_Test_ijk <- function(dt_policy_claim, date_i,j_dev_period, k, reserving_dates, model_vars) {
  
  # # debugging
  # #~~~~~~~~~~~~~
  # dt_policy_claim = dt_polclaim
  # date_i = t_i
  # j_dev_period = 1
  # k = 1
  # reserving_dates = lst_Date_slice
  # #~~~~~~~~~~~~~
  #   
  date_i <- as.Date(date_i)
  
  #i - j - k + 1 (predictor as at date)
  date_lookup <- (reserving_dates[which(reserving_dates == (date_i))]) 
  
  #i - k to calculate target incremental paid
  target_lookup <- (reserving_dates[which(reserving_dates == (date_i)) +j_dev_period - 1]) 
  
  #i -k + 1 to calculate target incremental paid  
  target_lookup_next <- (reserving_dates[which(reserving_dates == (date_i)) + j_dev_period]) 
  
  #definition of reported but not settled
  # P_te_RBNS rowids of policies needing an RBNS reserve
  dt_policy_claim <- dt_policy_claim[date_report <= date_lookup & date_lookup < date_pay] 
  
  #model assumes one payment so just need to check date paid
  dt_policy_claim[, ':='(date_lookup = date_lookup,
                         delay_train = as.numeric(date_lookup - date_pol_start), #extra feature
                         j = j_dev_period,
                         k = k,
                         target = ifelse(date_pay<=target_lookup,0,ifelse(date_pay<=target_lookup_next,claim_cost,0)))] 
  
return(dt_policy_claim[, model_vars, with = FALSE])
  
}


In [17]:

RBNS_Train <- function(dt_policy_claim, date_i, i, k, reserving_dates, model_vars) {
# Create a combined TRAIN dataset across all k and j combos
  for (k in 1:k){
    if (k==1) dt_train <- NULL
    for (j in 1:(i - k + 1)){
      dt_train <- rbind(dt_train, RBNS_Train_ijk(dt_polclaim, date_i, j, k,reserving_dates, model_vars))
    }
  }  
  return(dt_train)
}

In [18]:


RBNS_Test <- function(dt_policy_claim, date_i, delta, k, reserving_dates, model_vars) {
  
  # Create a combined TEST dataset across all k and j combos
  for (k in 1:k){
    if (k==1) dt_test <- NULL
    for (j in 1:(delta - k + 1)){
      dt_test <- rbind(dt_test, RBNS_Test_ijk(dt_polclaim, date_i, j, k,reserving_dates, model_vars))
    }
  }
  
  return(dt_test)
}


In [19]:
#define modelVars
RBNS_model_vars <- c("clm_number",
                     "pol_number",
                     "j",
                     "k",
                     "date_pol_start",
                     "date_occur",
                     "date_report",
                     "date_pay",
                     "Cover",
                     "claim_type",
                     "Brand",
                     "Model",
                     "Price",
                     "target"
    )


# Create a combined TRAIN dataset for k = 1 and all valid j delay values
dt_RBNS_train <- RBNS_Train(dt_polclaim, t_i, i, k = 1, lst_Date_slice, RBNS_model_vars)

# Create a combined TEST dataset for k = 1 and all valid j delay values
dt_RBNS_test <- RBNS_Test(dt_polclaim, t_i, delta, k = 1, lst_Date_slice, RBNS_model_vars)


In [52]:
head(dt_RBNS_train)


head(dt_RBNS_test)



clm_number,pol_number,j,k,date_pol_start,date_occur,date_report,date_pay,Cover,claim_type,Brand,Model,Price,target,flgTrain
<chr>,<chr>,<int>,<int>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201608040001,201601010103,1,1,2016-01-01,2016-07-15,2016-08-04,2016-09-02,B,B,1,3,913,201,1
201608110001,201601010130,1,1,2016-01-01,2016-05-11,2016-08-11,2016-09-14,B,B,2,2,728,448,1
201608170001,201601010141,1,1,2016-01-01,2016-08-02,2016-08-17,2016-09-16,B,B,1,3,913,15,1
201608280001,201601010145,1,1,2016-01-01,2016-08-28,2016-08-28,2016-09-24,B,B,1,3,913,437,1
201608190001,201601010190,1,1,2016-01-01,2016-08-19,2016-08-19,2016-09-21,BOT,T,2,2,728,725,1
201608130001,201601010194,1,1,2016-01-01,2016-06-23,2016-08-13,2016-09-11,BOT,B,2,1,633,118,1


clm_number,pol_number,j,k,date_pol_start,date_occur,date_report,date_pay,Cover,claim_type,Brand,Model,Price,target,flgTrain
<chr>,<chr>,<int>,<int>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201609150001,201601010014,1,1,2016-01-01,2016-09-15,2016-09-15,2016-10-17,B,B,2,3,837,309,0
201609090001,201601010025,1,1,2016-01-01,2016-09-09,2016-09-09,2016-10-07,B,B,1,2,794,357,0
201609220001,201601010212,1,1,2016-01-01,2016-09-02,2016-09-22,2016-10-20,BOT,T,2,3,837,837,0
201609020001,201601010261,1,1,2016-01-01,2016-06-06,2016-09-02,2016-10-01,BOT,T,1,1,690,675,0
201609060001,201601010284,1,1,2016-01-01,2016-08-25,2016-09-06,2016-10-09,BOT,T,1,3,913,908,0
201609200001,201601010327,1,1,2016-01-01,2016-09-20,2016-09-20,2016-10-26,BOT,T,1,2,794,666,0


In [21]:


# Add a flag to determine which rows are from the trainset and which from the test set
dt_RBNS_train[, flgTrain := 1]
dt_RBNS_test[, flgTrain := 0]

# combine into a single RBNS dataset   
dt_All_RBNS <- rbind(dt_RBNS_train, dt_RBNS_test)
#write.csv(dt_All_RBNS,"dt_All_RBNS.csv", row.names = F)
    


In [53]:
head(dt_RBNS_train)
head(dt_RBNS_test)

cat("Rows:", nrow(dt_RBNS_train), " Columns:", ncol(dt_RBNS_train), "\n")
cat("Rows:", nrow(dt_RBNS_test), " Columns:", ncol(dt_RBNS_test), "\n")


clm_number,pol_number,j,k,date_pol_start,date_occur,date_report,date_pay,Cover,claim_type,Brand,Model,Price,target,flgTrain
<chr>,<chr>,<int>,<int>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201608040001,201601010103,1,1,2016-01-01,2016-07-15,2016-08-04,2016-09-02,B,B,1,3,913,201,1
201608110001,201601010130,1,1,2016-01-01,2016-05-11,2016-08-11,2016-09-14,B,B,2,2,728,448,1
201608170001,201601010141,1,1,2016-01-01,2016-08-02,2016-08-17,2016-09-16,B,B,1,3,913,15,1
201608280001,201601010145,1,1,2016-01-01,2016-08-28,2016-08-28,2016-09-24,B,B,1,3,913,437,1
201608190001,201601010190,1,1,2016-01-01,2016-08-19,2016-08-19,2016-09-21,BOT,T,2,2,728,725,1
201608130001,201601010194,1,1,2016-01-01,2016-06-23,2016-08-13,2016-09-11,BOT,B,2,1,633,118,1


clm_number,pol_number,j,k,date_pol_start,date_occur,date_report,date_pay,Cover,claim_type,Brand,Model,Price,target,flgTrain
<chr>,<chr>,<int>,<int>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201609150001,201601010014,1,1,2016-01-01,2016-09-15,2016-09-15,2016-10-17,B,B,2,3,837,309,0
201609090001,201601010025,1,1,2016-01-01,2016-09-09,2016-09-09,2016-10-07,B,B,1,2,794,357,0
201609220001,201601010212,1,1,2016-01-01,2016-09-02,2016-09-22,2016-10-20,BOT,T,2,3,837,837,0
201609020001,201601010261,1,1,2016-01-01,2016-06-06,2016-09-02,2016-10-01,BOT,T,1,1,690,675,0
201609060001,201601010284,1,1,2016-01-01,2016-08-25,2016-09-06,2016-10-09,BOT,T,1,3,913,908,0
201609200001,201601010327,1,1,2016-01-01,2016-09-20,2016-09-20,2016-10-26,BOT,T,1,2,794,666,0


Rows: 11764  Columns: 15 
Rows: 33450  Columns: 15 


In [23]:
write.csv(dt_RBNS_train, "dt_RBNS_train.csv", row.names = FALSE)
write.csv(dt_RBNS_test, "dt_RBNS_test.csv", row.names = FALSE)


In [54]:

head(dt_All_RBNS)
cat("Rows:", nrow(dt_All_RBNS), " Columns:", ncol(dt_All_RBNS), "\n")

clm_number,pol_number,j,k,date_pol_start,date_occur,date_report,date_pay,Cover,claim_type,Brand,Model,Price,target,flgTrain
<chr>,<chr>,<int>,<int>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201608040001,201601010103,1,1,2016-01-01,2016-07-15,2016-08-04,2016-09-02,B,B,1,3,913,201,1
201608110001,201601010130,1,1,2016-01-01,2016-05-11,2016-08-11,2016-09-14,B,B,2,2,728,448,1
201608170001,201601010141,1,1,2016-01-01,2016-08-02,2016-08-17,2016-09-16,B,B,1,3,913,15,1
201608280001,201601010145,1,1,2016-01-01,2016-08-28,2016-08-28,2016-09-24,B,B,1,3,913,437,1
201608190001,201601010190,1,1,2016-01-01,2016-08-19,2016-08-19,2016-09-21,BOT,T,2,2,728,725,1
201608130001,201601010194,1,1,2016-01-01,2016-06-23,2016-08-13,2016-09-11,BOT,B,2,1,633,118,1


Rows: 45214  Columns: 15 


In [25]:
IBNR_Freq_Train_ijk <- function(dt_policy_claim, date_i, j_dev_period, k, reserving_dates, model_vars, verbose = FALSE) {
  
  date_i <- as.Date(date_i)
  date_k <- (reserving_dates[which(reserving_dates == date_i) - k + 1])
  date_j <- (reserving_dates[which(reserving_dates == date_k) - j_dev_period])
  date_lookup <- (reserving_dates[which(reserving_dates == (date_i)) - j_dev_period -k + 1])
  target_lookup <- (reserving_dates[which(reserving_dates == (date_i)) - k])
  target_lookup_next <- (reserving_dates[which(reserving_dates == (date_i)) - k + 1])
  
  if(verbose) cat(paste("Valn date", date_i, ", j = ", j_dev_period, ", k =", k, "\n"))
  
  dt_policy_claim <- dt_policy_claim[date_pol_start < date_lookup  & date_lookup < date_report]
  
  dt_policy_claim[, ':='(
    date_i = date_i,
    date_lookup = date_lookup,
    target_lookup = target_lookup,
    target_lookup_next = target_lookup_next,
    delay_train = as.numeric(date_lookup - date_pol_start),
    j = j_dev_period,
    k = k,
    exposure = round((pmin(as.numeric(as.numeric(date_pol_end)), as.numeric(floor_date(date_i, unit= "second"))) - as.numeric(date_pol_start))/(24*60*60*365), 3),
    target = ifelse(target_lookup <= date_pay &  date_pay < target_lookup_next & date_occur <= date_lookup ,1,0)
  )]
  
  # Now include the new variables in the output:
  all_vars <- unique(c(model_vars, "date_i", "date_lookup", "target_lookup", "target_lookup_next", "delay_train"))
  
  dt_policy_claim <- dt_policy_claim[, ..all_vars]
  
  return(dt_policy_claim)
}


In [26]:
IBNR_Loss_Train_ijk <- function(dt_policy_claim, date_i, j_dev_period, k, reserving_dates, model_vars, verbose = FALSE) {

  date_i <- as.Date(date_i)
  date_k <- reserving_dates[which(reserving_dates == date_i) - k + 1]
  date_j <- reserving_dates[which(reserving_dates == date_k) - j_dev_period]
  date_lookup <- reserving_dates[which(reserving_dates == date_i) - j_dev_period - k + 1]
  target_lookup <- reserving_dates[which(reserving_dates == date_i) - k]
  target_lookup_next <- reserving_dates[which(reserving_dates == date_i) - k + 1]

  if(verbose) cat(paste("Valn date", date_i, ", j = ", j_dev_period, ", k =", k, "\n"))

  dt_policy_claim <- dt_policy_claim[
    (date_lookup < date_report) &
    (date_occur < date_lookup) &
    (target_lookup <= date_pay & date_pay < target_lookup_next)
  ]

  dt_policy_claim[, ':='(
    date_i = date_i,
    date_k = date_k,
    date_j = date_j,
    date_lookup = date_lookup,
    target_lookup = target_lookup,
    target_lookup_next = target_lookup_next,
    delay_train = as.numeric(date_lookup - date_pol_start),
    j = j_dev_period,
    k = k,
    exposure = 1,
    target = ifelse(target_lookup <= date_pay & date_pay < target_lookup_next, claim_cost, 0)
  )]

  # Include extra variables along with model_vars
  all_vars <- unique(c(model_vars, "date_i", "date_k", "date_j", "date_lookup", "target_lookup", "target_lookup_next", "delay_train"))

  # Return subset with all these variables
  return(dt_policy_claim[, ..all_vars])
}


In [27]:
IBNR_Test_ijk <- function(dt_policy_claim, date_i, j_dev_period, k, reserving_dates, model_vars, verbose = FALSE) {

  date_i <- as.Date(date_i)
  date_lookup <- reserving_dates[which(reserving_dates == date_i)]  # predictor as at date
  target_lookup <- reserving_dates[which(reserving_dates == date_i) + j_dev_period - 1]
  target_lookup_next <- reserving_dates[which(reserving_dates == date_i) + j_dev_period]

  if(verbose) cat(paste("Valn date", date_i, ", j = ", j_dev_period, ", k =", k, "\n"))

  dt_policy_claim <- dt_policy_claim[date_pol_start <= date_lookup & date_lookup < date_report]  # IBNR

  dt_policy_claim[, ':='(
    date_i = date_i,
    date_lookup = date_lookup,
    target_lookup = target_lookup,
    target_lookup_next = target_lookup_next,
    delay_train = as.numeric(date_lookup - date_pol_start),  # extra feature
    j = j_dev_period,
    k = k,
    exposure = round(
      (pmin(as.numeric(as.numeric(date_pol_end)), as.numeric(floor_date(date_i, unit = "second"))) -
       as.numeric(date_pol_start)) / (24 * 60 * 60 * 365), 3),
    target = ifelse(target_lookup <= date_pay & date_pay < target_lookup_next & date_occur <= date_lookup, claim_cost, 0)
  )]

  # Include extra variables in the returned dataset
  all_vars <- unique(c(model_vars, "date_i", "date_lookup", "target_lookup", "target_lookup_next", "delay_train"))

  return(dt_policy_claim[, ..all_vars])
}


In [55]:
# Initialize storage lists
freq_list <- list()
loss_list <- list()
test_list <- list()

# Set k and loop over valid j values
k <- 1
for (j in 1:(i - k + 1)) {
  
  # Freq
  freq_out <- IBNR_Freq_Train_ijk(dt_polclaim, t_i, j, k, lst_Date_slice, IBNR_model_vars)
  freq_out[, j_used := j]
  freq_list[[j]] <- freq_out
  
  # Loss
  loss_out <- IBNR_Loss_Train_ijk(dt_polclaim, t_i, j, k, lst_Date_slice, IBNR_model_vars)
  loss_out[, j_used := j]
  loss_list[[j]] <- loss_out
  
  # Test
  if (j <= (delta - k + 1)) {
    test_out <- IBNR_Test_ijk(dt_polclaim, t_i, j, k, lst_Date_slice, IBNR_model_vars)
    test_out[, j_used := j]
    test_list[[j]] <- test_out
  }
}

# Combine outputs
all_freq <- rbindlist(freq_list, fill = TRUE)
all_loss <- rbindlist(loss_list, fill = TRUE)
all_test <- rbindlist(test_list, fill = TRUE)

all_freq[, flgTrain := 1]   # Frequency training data
all_loss[, flgTrain := 2]   # Loss training data
all_test[, flgTrain := 0]  

# Display top rows (first 10 each)

head(all_freq)
cat("Rows:", nrow(all_freq), " Columns:", ncol(all_freq), "\n")

head(all_loss)
cat("Rows:", nrow(all_loss), " Columns:", ncol(all_loss), "\n")

head(all_test)
cat("Rows:", nrow(all_test), " Columns:", ncol(all_test), "\n")
# Write to CSV
#write.csv(all_freq, "IBNR_Freq_Train_ijk_all.csv", row.names = FALSE)
#write.csv(all_loss, "IBNR_Loss_Train_ijk_all.csv", row.names = FALSE)
#write.csv(all_test, "IBNR_Test_ijk_all.csv", row.names = FALSE)


clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,Model,Price,target,date_i,date_lookup,target_lookup,target_lookup_next,delay_train,j_used,flgTrain
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,...,<dbl>,<dbl>,<dbl>,<date>,<dttm>,<dttm>,<dttm>,<dbl>,<int>,<dbl>
NA,201601010002,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1
NA,201601010003,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1
NA,201601010004,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1
NA,201601010005,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1
NA,201601010006,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1
NA,201601010007,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-08-28,2016-08-28,2016-09-27,240,1,1


Rows: 732120  Columns: 21 


clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,target,date_i,date_k,date_j,date_lookup,target_lookup,target_lookup_next,delay_train,j_used,flgTrain
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,...,<dbl>,<date>,<dttm>,<dttm>,<dttm>,<dttm>,<dttm>,<dbl>,<int>,<dbl>
201608290002,201601090176,1,1,1,2016-01-09,2016-08-14,2016-08-29,2016-09-25,BOT,...,223,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,232,1,2
201608300007,201601150402,1,1,1,2016-01-15,2016-08-27,2016-08-30,2016-09-25,BO,...,51,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,226,1,2
201608310016,201601160419,1,1,1,2016-01-16,2016-08-17,2016-08-31,2016-09-20,BO,...,296,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,225,1,2
201609020008,201601180292,1,1,1,2016-01-18,2016-08-24,2016-09-02,2016-09-22,BOT,...,101,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,223,1,2
201608290008,201601190489,1,1,1,2016-01-19,2016-08-26,2016-08-29,2016-09-22,BO,...,624,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,222,1,2
201609020010,201601200445,1,1,1,2016-01-20,2016-08-26,2016-09-02,2016-09-21,BO,...,152,2016-09-27,2016-09-27,2016-08-28,2016-08-28,2016-08-28,2016-09-27,221,1,2


Rows: 1006  Columns: 23 


clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,Model,Price,target,date_i,date_lookup,target_lookup,target_lookup_next,delay_train,j_used,flgTrain
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<chr>,...,<dbl>,<dbl>,<dbl>,<date>,<dttm>,<dttm>,<dttm>,<dbl>,<int>,<dbl>
NA,201601010002,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0
NA,201601010003,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0
NA,201601010004,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0
NA,201601010005,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0
NA,201601010006,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0
NA,201601010007,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,2016-09-27,2016-09-27,2016-09-27,2016-10-27,23328000,1,0


Rows: 1778580  Columns: 21 


In [34]:
# Load data.table functions (if not already loaded)
library(data.table)
library(lubridate)

# Set key and calculate delays for FREQ
setkey(all_freq, clm_number, k, j)
all_freq[, Count := .N , by = clm_number]
all_freq[, ':='(
  delay_uw_occ = ifelse(year(date_occur) == 2199, -1,
                        ceiling((as.numeric(date_occur) - as.numeric(date_pol_start)) / (24*60*60))),
  
  delay_occ_rep = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_report) - as.numeric(date_occur)) / (24*60*60))),
  
  delay_rep_pay = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_pay) - as.numeric(date_report)) / (24*60*60))),
  
  delay_uw_val = ceiling((as.numeric(t_i) - as.numeric(date_pol_start)) / (24*60*60)),
  date_uw = ceiling(as.numeric(date_pol_start) / (24*60*60)),
  Cover = as.factor(Cover)
)]

# Repeat for LOSS
setkey(all_loss, clm_number, k, j)
all_loss[, Count := .N , by = clm_number]
all_loss[, ':='(
  delay_uw_occ = ifelse(year(date_occur) == 2199, -1,
                        ceiling((as.numeric(date_occur) - as.numeric(date_pol_start)) / (24*60*60))),
  
  delay_occ_rep = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_report) - as.numeric(date_occur)) / (24*60*60))),
  
  delay_rep_pay = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_pay) - as.numeric(date_report)) / (24*60*60))),
  
  delay_uw_val = ceiling((as.numeric(t_i) - as.numeric(date_pol_start)) / (24*60*60)),
  date_uw = ceiling(as.numeric(date_pol_start) / (24*60*60)),
  Cover = as.factor(Cover)
)]

# Repeat for TEST
setkey(all_test, clm_number, k, j)
all_test[, Count := .N , by = clm_number]
all_test[, ':='(
  delay_uw_occ = ifelse(year(date_occur) == 2199, -1,
                        ceiling((as.numeric(date_occur) - as.numeric(date_pol_start)) / (24*60*60))),
  
  delay_occ_rep = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_report) - as.numeric(date_occur)) / (24*60*60))),
  
  delay_rep_pay = ifelse(year(date_occur) == 2199, -1,
                         ceiling((as.numeric(date_pay) - as.numeric(date_report)) / (24*60*60))),
  
  delay_uw_val = ceiling((as.numeric(t_i) - as.numeric(date_pol_start)) / (24*60*60)),
  date_uw = ceiling(as.numeric(date_pol_start) / (24*60*60)),
  Cover = as.factor(Cover)
)]


In [41]:
#head(all_freq)

#head(all_loss)

head(all_test)

clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,target_lookup_next,delay_train,j_used,flgTrain,Count,delay_uw_occ,delay_occ_rep,delay_rep_pay,delay_uw_val,date_uw
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<fct>,...,<dttm>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,201601010002,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010003,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010004,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010005,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010006,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010007,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801


In [42]:
total_rows <- nrow(all_test)
cat("Total number of rows:", total_rows, "\n")


Total number of rows: 1778580 


In [43]:

# Create a new table without duplicates
all_test_unique <- unique(all_test, by = c("clm_number", "pol_number"))

# Check result
head(all_test_unique)


clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,target_lookup_next,delay_train,j_used,flgTrain,Count,delay_uw_occ,delay_occ_rep,delay_rep_pay,delay_uw_val,date_uw
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<fct>,...,<dttm>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,201601010002,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010003,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010004,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010005,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010006,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801
NA,201601010007,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,2016-10-27,23328000,1,0,1479120,-1,-1,-1,270,16801


In [46]:
all_test_nonzero <- all_test[target != 0]

# View first few rows
head(all_test_nonzero)


clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,target_lookup_next,delay_train,j_used,flgTrain,Count,delay_uw_occ,delay_occ_rep,delay_rep_pay,delay_uw_val,date_uw
<chr>,<chr>,<int>,<dbl>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<fct>,...,<dttm>,<dbl>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
201609280001,201601030069,2,1,0.734,2016-01-03,2016-09-19,2016-09-28,2016-11-01,B,...,2016-11-26,23155200,2,0,10,260,9,34,268,16803
201609280004,201601080227,2,1,0.721,2016-01-08,2016-09-26,2016-09-28,2016-10-30,BOT,...,2016-11-26,22723200,2,0,10,262,2,32,263,16808
201609280005,201601150682,2,1,0.701,2016-01-15,2016-09-27,2016-09-28,2016-10-27,BO,...,2016-11-26,22118400,2,0,10,256,1,29,256,16815
201609280006,201601170365,1,1,0.696,2016-01-17,2016-09-20,2016-09-28,2016-10-23,BOT,...,2016-10-27,21945600,1,0,10,247,8,25,254,16817
201609280007,201601170637,1,1,0.696,2016-01-17,2016-09-16,2016-09-28,2016-10-21,BO,...,2016-10-27,21945600,1,0,10,243,12,23,254,16817
201609280008,201601180671,1,1,0.693,2016-01-18,2016-08-27,2016-09-28,2016-10-18,BO,...,2016-10-27,21859200,1,0,10,222,32,20,253,16818


In [47]:
total_rows <- nrow(all_test_nonzero)
cat("Total number of rows:", total_rows, "\n")

Total number of rows: 1516 


In [44]:
total_rows_unique <- nrow(all_test_unique)
cat("Total number of rows:", total_rows, "\n")

Total number of rows: 1778580 


In [45]:
total_target_sum <- sum(all_test_unique$target, na.rm = TRUE)
cat("Sum of target column:", total_target_sum, "\n")


Sum of target column: 37980 


In [36]:
data.table(
  Dataset = c("Freq Train", "Loss Train", "Test"),
  Rows = c(nrow(all_freq), nrow(all_loss), nrow(all_test))
)


Dataset,Rows
<chr>,<int>
Freq Train,732120
Loss Train,1006
Test,1778580


In [37]:
data.table(
  Dataset = c("Freq Train", "Loss Train", "Test"),
  Rows_with_NA_clm_number = c(
    sum(is.na(all_freq$clm_number)),
    sum(is.na(all_loss$clm_number)),
    sum(is.na(all_test$clm_number))
  )
)



Dataset,Rows_with_NA_clm_number
<chr>,<int>
Freq Train,588668
Loss Train,0
Test,1479120


In [38]:
write.csv(all_loss, "IBNR_Loss_Train_ijk_all.csv", row.names = FALSE)

In [39]:
write.csv(all_freq, "IBNR_Freq_Train_ijk_all.csv", row.names = FALSE)


In [40]:
write.csv(all_test, "IBNR_Test_ijk_all.csv", row.names = FALSE)

In [29]:


IBNR_Train <- function(dt_policy_claim, date_i, i, k, reserving_dates, model_vars, verbose = FALSE) {

  # Create a combined TRAIN dataset across all k and j combos
    for (k in 1:k){
      if (k==1){
        dt_train_Freq <- NULL
        dt_train_Loss <- NULL
      }
      
      for (j in 1:(i - k + 1)){
        dt_train_Freq <- rbind(dt_train_Freq, IBNR_Freq_Train_ijk(dt_policy_claim, date_i, j, k,reserving_dates, model_vars, verbose))
        dt_train_Loss <- rbind(dt_train_Loss, IBNR_Loss_Train_ijk(dt_policy_claim, date_i, j, k,reserving_dates, model_vars, verbose))
      }
    }

  return(list(Freq = dt_train_Freq, Loss = dt_train_Loss))
}


In [30]:
IBNR_Test <- function(dt_policy_claim, date_i, delta, k, reserving_dates, model_vars, verbose = FALSE) {
 
  # Create a combined TEST dataset across all k and j combos
  for (k in 1:k){
    if (k==1) dt_test <- NULL
    for (j in 1:(delta - k + 1)){
      dt_test <- rbind(dt_test, IBNR_Test_ijk(dt_policy_claim, date_i, j, k,reserving_dates, model_vars, verbose))
    }
  }
  return(dt_test)
}


In [31]:
#define IBNR modelVars
IBNR_model_vars <- c("clm_number",
                     "pol_number",
                     "j",
                     "k",
                     "exposure",
                     "date_pol_start",
                     "date_occur",
                     "date_report",
                     "date_pay",
                     "Cover",
                     "Brand",
                     "Model",
                     "Price",
                     "target")
    
# Create a combined TRAIN dataset for k = 1 and all valid j delay values
lst_IBNR_train <- IBNR_Train(dt_polclaim, t_i, i, k = 1,lst_Date_slice, IBNR_model_vars)

# Create a combined TEST dataset for k = 1 and all valid j delay values
dt_IBNR_test <- IBNR_Test(dt_polclaim, t_i, delta, k = 1,lst_Date_slice, IBNR_model_vars)

In [32]:
# Set flags
lst_IBNR_train$Freq[, flgTrain := 1]
lst_IBNR_train$Loss[, flgTrain := 2]
dt_IBNR_test[, flgTrain := 0]

# Combine datasets
dt_All_IBNR <- rbind(
  lst_IBNR_train$Freq,
  lst_IBNR_train$Loss,
  dt_IBNR_test
)

# Optional: Write to CSV
# write.csv(dt_All_IBNR, "dt_ALL_IBNR.csv", row.names = FALSE)


ERROR: Error in rbindlist(l, use.names, fill, idcol): Item 2 has 22 columns, inconsistent with item 1 which has 20 columns. To fill missing columns use fill=TRUE.


In [61]:
# order and create some delay fields
setkey(dt_All_IBNR, clm_number, k, j)
    
dt_All_IBNR[, Count := .N , by =clm_number]
dt_All_IBNR[,':='( delay_uw_occ = ifelse(year(date_occur) == 2199,
                                        -1,
                                        ceiling((as.numeric(date_occur) - as.numeric(date_pol_start))
                                                  /(24*60*60))
                                          ),
                   delay_occ_rep = ifelse(year(date_occur) == 2199,
                                          -1,
                                          ceiling((as.numeric(date_report) - as.numeric(date_occur))
                                                  /(24*60*60))
                                          ),
                   delay_rep_pay = ifelse(year(date_occur) == 2199,
                                          -1,
                                          ceiling((as.numeric(date_pay) - as.numeric(date_report))
                                                  /(24*60*60))
                                          ),
                   delay_uw_val = ceiling((as.numeric(t_i) - as.numeric(date_pol_start))/(24*60*60)),
                   date_uw = ceiling(as.numeric(date_pol_start)/(24*60*60)),
                   Cover = as.factor(Cover))]

In [62]:
head(dt_All_IBNR)

clm_number,pol_number,j,k,exposure,date_pol_start,date_occur,date_report,date_pay,Cover,...,Model,Price,target,flgTrain,Count,delay_uw_occ,delay_occ_rep,delay_rep_pay,delay_uw_val,date_uw
<chr>,<chr>,<int>,<int>,<dbl>,<dttm>,<dttm>,<dttm>,<dttm>,<fct>,...,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NA,201601010002,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
NA,201601010003,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
NA,201601010004,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
NA,201601010005,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
NA,201601010006,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
NA,201601010007,1,1,0.74,2016-01-01,2199-12-31 23:59:59,2199-12-31 23:59:59,2199-12-31 23:59:59,B,...,3,913,0,1,2067788,-1,-1,-1,270,16801
